In [1]:
import pandas as pd
import numpy as np
import math
import statsmodels.api as sm

starting_year_to_filter = 1963
end_year_to_filter = 2020
number_of_lookback_periods = 120
data_to_read_address = "data/25_Portfolios_5x5_size_value_monthly.CSV"
factor_data_address = "data/F-F_Research_Data_5_Factors_2x3.csv"
number_of_PPs_to_consider = 3
number_of_PEPs_to_consider = 3
number_of_PAPs_to_consider = 3

def rank_and_map(df):
    # Make a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    # Exclude the 'date' column for ranking
    data_columns = df_copy.columns[1:]
    
    # Apply ranking and scaling row-wise (for each date)
    def rank_row(row):
        # Get the ranks (min rank is 1)
        ranks = row.rank(method='min')
        # Normalize the ranks to range between 0 and 1
        ranks_normalized = (ranks - 1) / (len(row) - 1)
        # Map to range [-0.5, 0.5]
        return ranks_normalized - 0.5
    
    # Apply rank_row function to each row, excluding the 'date' column
    df_copy[data_columns] = df_copy[data_columns].apply(rank_row, axis=1)
    return df_copy


def cross_sectional_demean(df):
    # Make a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    # Exclude the 'date' column
    data_columns = df_copy.columns[1:]
    
    # Apply demeaning row-wise (for each date)
    def demean_row(row):
        row_mean = row.mean()  # Compute the mean of the row
        return row - row_mean  # Subtract the mean from each element in the row
    
    # Apply demean_row function to each row, excluding the 'date' column
    df_copy[data_columns] = df_copy[data_columns].apply(demean_row, axis=1)
    return df_copy


def compute_rs_product(df1, df2):
    # Ensure the date columns match
    if not df1['date'].equals(df2['date']):
        raise ValueError("Date columns of both dataframes must match.")
    
  # Convert to numeric, set invalid values as NaN
    df1 = df1.astype({col: 'float64' for col in df1.columns if col != 'date'})
    df2 = df2.astype({col: 'float64' for col in df2.columns if col != 'date'})
    result = {}
    
    # Iterate over each row (each date)
    for index, date in enumerate(df1['date']):
        # Get the R vector (from df1) and S' vector (from df2) for the current date
        R = df1.iloc[index, 1:].values.reshape(-1, 1)  # n x 1 vector
        S_transpose = df2.iloc[index, 1:].values.reshape(1, -1)  # 1 x n vector
        # Compute the outer product (RS')
        matrix_rs = np.dot(R, S_transpose)  # n x n matrix
        # Store the result in a dictionary, with date as the key
        result[date] = matrix_rs

    return result


def get_prediction_matrix(input_date, result_matrices, n_periods):
    # Sort the dates in result_matrices to ensure they're in order
    sorted_dates = sorted(result_matrices.keys())
    # Find the index of the input date in the sorted list of dates
    if input_date not in sorted_dates:
        raise ValueError("The input date is not found in the result_matrices.")
    
    input_date_index = sorted_dates.index(input_date)
    # Select the last n_periods (excluding the input date)
    start_index = max(0, input_date_index - n_periods)  # Ensure we don't go below index 0
    selected_dates = sorted_dates[start_index:input_date_index]
    
    if len(selected_dates) == 0:
        raise ValueError(f"There are no previous periods to calculate the average for the given number: {n_periods}.")
    
    # Initialize a matrix to accumulate the sum
    matrix_shape = result_matrices[sorted_dates[0]].shape
    sum_matrix = np.zeros(matrix_shape, dtype=float)
    # Sum all the selected matrices
    for date in selected_dates:
        sum_matrix += np.array(result_matrices[date], dtype=float)
    
    # Calculate the element-wise average
    average_matrix = sum_matrix / len(selected_dates)
    return average_matrix


# i should start from 0. In other words, to get the first PP's expected return you must set i=0.
def get_ith_PPs_expected_return(S,i):
    return S[i]

# i should start from 0. In other words, to get the first PP you must set i=0.
def get_ith_position_matrix(U,VT,i):
    u_column = U[:, i]
    v_column = VT[i, :]
    return np.outer(v_column,u_column)

def first_n_PPs_expected_return(S,n):
    sum = 0
    for i in range(n):
        sum += get_ith_PPs_expected_return(S,i)
    return sum

def first_n_PPs_position_matrix(U,VT,number_of_PPs):
    matrix_shape = U.shape
    sum_matrix = np.zeros(matrix_shape, dtype=float)
    for i in range(number_of_PPs):
        sum_matrix += get_ith_position_matrix(U,VT,i)
    return sum_matrix/number_of_PPs


# i should start from 0. In other words, to get the first PEP you must set i=0.
def get_ith_PEPs_expected_return(eigenvalues,i):
    return eigenvalues[i]

def get_ith_symmetric_position_matrix(eigenvectors,i):
    w = eigenvectors[:, i]
    return np.outer(w,w)

def first_n_PEPs_expected_return(eigenvalues,n):
    sum = 0
    for i in range(n):
        sum += abs(get_ith_PEPs_expected_return(eigenvalues,i))
    return sum

def first_n_PEPs_position_matrix(eigenvectors,number_of_PEPs):
    matrix_shape = eigenvectors.shape
    sum_matrix = np.zeros(matrix_shape, dtype=float)
    for i in range(number_of_PEPs):
        sum_matrix += get_ith_symmetric_position_matrix(eigenvectors,i)
    return sum_matrix/number_of_PEPs

# i should start from 0. In other words, to get the first PEP you must set i=0.
def get_ith_PAPs_expected_return(filtered_eigenvalues_ta,i):
    return 2 * filtered_eigenvalues_ta[i]

def get_ith_asymmetric_position_matrix(sorted_eigenvectors_ta_real_part,sorted_eigenvectors_ta_imaginary_part,i):
    return np.outer(sorted_eigenvectors_ta_real_part[:,i],sorted_eigenvectors_ta_imaginary_part[:,i]) - np.outer(sorted_eigenvectors_ta_imaginary_part[:,i],sorted_eigenvectors_ta_real_part[:,i])
    
def first_n_PAPs_expected_return(filtered_eigenvalues_ta,n):
    sum = 0
    for i in range(n):
        sum += get_ith_PAPs_expected_return(filtered_eigenvalues_ta,i)
    return sum

def first_n_PAPs_position_matrix(sorted_eigenvectors_ta_real_part,sorted_eigenvectors_ta_imaginary_part,number_of_PAPs):
    number_of_rows = sorted_eigenvectors_ta_real_part.shape[0]
    sum_matrix = np.zeros((number_of_rows,number_of_rows), dtype=float)
    for i in range(number_of_PAPs):
        sum_matrix += get_ith_asymmetric_position_matrix(sorted_eigenvectors_ta_real_part,sorted_eigenvectors_ta_imaginary_part,i)
    return sum_matrix/number_of_PAPs

def calculate_sharpe_ratio(returns):
    # Compute excess returns
    
    # Compute average excess return
    average_return = returns.mean()
    
    # Compute standard deviation of returns
    std_dev_returns = returns.std()
    
    # Compute Sharpe Ratio
    sharpe_ratio = average_return / std_dev_returns
    
    return sharpe_ratio

def filter_dataframes_by_common_dates(df1, df2):
    # Find common dates (intersection of index values)
    common_dates = df1.index.intersection(df2.index)
    
    # Filter both dataframes to keep only the rows with the common dates
    df1_filtered = df1.loc[common_dates]
    df2_filtered = df2.loc[common_dates]
    
    return df1_filtered, df2_filtered


def regression_results(X, Y):
    # Get the standard deviation of X[1] (which is the first non-constant column of X)
    std_X1 = X.iloc[:, 1].std()  # Assuming X[1] is the first non-constant column

    # Scale Y to have the same standard deviation as X[1]
    std_Y = Y.std()
    Y_scaled = Y * (std_X1 / std_Y)  # Scale Y by the ratio of std_X1 to std_Y

    # Add a constant (intercept) to the independent variables
    X = sm.add_constant(X)

    # Fit the model
    model = sm.OLS(Y_scaled, X).fit()

    # Get the coefficients (including the intercept)
    coefficients = model.params.values

    # Get the t-statistics
    t_stats = model.tvalues.values

    # Calculate the standard deviation of residuals
    residuals = model.resid
    std_residuals = residuals.std(ddof=1)  # Sample standard deviation (ddof=1)

    # Calculate intercept divided by standard deviation of residuals
    intercept_over_std_residuals = (coefficients[0] / std_residuals) * math.sqrt(12) #sqrt(12) is to make the IR annualized. again, to make comparison with Kelly's paper easier.

    # Get the R-squared value
    r_squared = model.rsquared

    coefficients[0] = coefficients[0] * 12 # This is to make the results comparable to Kelly(2022). They report annualized alphas in their tables.

    # Return the list as required
    return [coefficients, t_stats, intercept_over_std_residuals, r_squared]


factor_data_monthly = pd.read_csv(factor_data_address)
factor_data_monthly['date'] = pd.to_datetime(factor_data_monthly['date'], format='%Y%m') + pd.offsets.MonthEnd(1)
factor_data_monthly = factor_data_monthly.set_index("date")
factor_data_monthly.head(5)


,Mkt-RF,SMB,HML,RMW,CMA,RF
date,,,,,,
1963-07-31,-0.39,-0.41,-0.97,0.68,-1.18,0.27
1963-08-31,5.07,-0.80,1.80,0.36,-0.35,0.25
1963-09-30,-1.57,-0.52,0.13,-0.71,0.29,0.27
1963-10-31,2.53,-1.39,-0.10,2.80,-2.01,0.29
1963-11-30,-0.85,-0.88,1.75,-0.51,2.24,0.27


In [ ]:
def build_PP(data_to_read_address, factor_data_monthly):
    
    df_25_ff_size_value_sorted_monthly = pd.read_csv(data_to_read_address)
    df_25_ff_size_value_sorted_monthly['date'] = pd.to_datetime(df_25_ff_size_value_sorted_monthly['date'], format='%Y%m') + pd.offsets.MonthEnd(1)
    
    signal_df = pd.DataFrame()
    signal_df["date"] = df_25_ff_size_value_sorted_monthly["date"]
    # Note that I shift signals one period forward to make computations easier. 
    signal_df= signal_df.join(df_25_ff_size_value_sorted_monthly.iloc[:, 1:].shift(1))
    # I can think of this matrix as $S_{t-1}$.
    normalized_signal_df = rank_and_map(signal_df)
    normalized_signal_df = normalized_signal_df[(normalized_signal_df['date'].dt.year > starting_year_to_filter) & (normalized_signal_df['date'].dt.year < end_year_to_filter)].reset_index(drop=True)
    # This matrix can be denoted as $R_{t-1}$
    demeaned_return_df = cross_sectional_demean(df_25_ff_size_value_sorted_monthly)
    demeaned_return_df = demeaned_return_df[(demeaned_return_df['date'].dt.year > starting_year_to_filter) & (demeaned_return_df['date'].dt.year < end_year_to_filter)].reset_index(drop=True)
    # This gives: $R_{t}S'_{t}$
    rs_matrix = compute_rs_product(demeaned_return_df, normalized_signal_df)
    
    """
    Prediction matrix for date T+1, used returns data up to month T and signals data up to month T-1.
    In the function get_prediction_matrix, I start the calculations from the previous month. 
    Note that although the input date is the current data, but the in the function that month is excluded.
    Note that in calculating realized returns, I am using the current month(the month of rearlized returns) as index. 
    But remember that the matrix was $S_{t-1}$. So, the index actually retreives the value of the previous month. 
    I formed the matrix this way in order to make the calculations easier.
    """

    realized_returns_df = pd.DataFrame(columns=[
        "date",
        "return_of_simple_factor", 
        "realized_return_of_first_three_PP", 
        "expected_return_of_first_three_PP",
        "realized_return_of_first_three_PEP",
        "expected_return_of_first_three_PEP",
        "realized_return_of_first_three_PAP",
        "expected_return_of_first_three_PAP"
    ])

    # I leave out the first 120 (number of lookback periods) observations to compute the prediction matrix.
    for date_index in demeaned_return_df.iloc[number_of_lookback_periods:]['date']:
        date_to_consider = pd.Timestamp(date_index)
        
        #for PP's
        prediction_matrix = get_prediction_matrix(date_to_consider, rs_matrix, number_of_lookback_periods)
        U, S, VT = np.linalg.svd(prediction_matrix)

        #for PEP's
        Symmetric_prediction_matrix = (prediction_matrix + prediction_matrix.T)/2
        eigenvalues, eigenvectors = np.linalg.eig(Symmetric_prediction_matrix)
        idx = eigenvalues.argsort()[::-1]  # Sort in descending order
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]

        # for PAP'S
        assymetric_prediction_matrix = 0.5 * (prediction_matrix - prediction_matrix.T)
        transposed_assymetric_prediction_matrix = assymetric_prediction_matrix.T
        eigenvalues_ta, eigenvectors_ta = np.linalg.eig(transposed_assymetric_prediction_matrix)
        sorted_indices_ta = np.argsort(-eigenvalues_ta.imag)
        sorted_eigenvalues_ta = eigenvalues_ta[sorted_indices_ta].imag
        sorted_eigenvectors_ta = eigenvectors_ta[:, sorted_indices_ta] * math.sqrt(2)  #sqrt(2) is to make the size of the vectors equal to 1.
        positive_indices = np.where(sorted_eigenvalues_ta > 0)
        filtered_eigenvalues_ta = sorted_eigenvalues_ta[positive_indices]
        filtered_eigenvectors_ta = sorted_eigenvectors_ta[:, positive_indices].squeeze()
        sorted_eigenvectors_ta_imaginary_part = filtered_eigenvectors_ta.imag
        sorted_eigenvectors_ta_real_part = filtered_eigenvectors_ta.real

        #to calculate realized returns
        signal_vector = normalized_signal_df[normalized_signal_df.date == date_to_consider].values[0, 1:].reshape(1, -1)  # 1*n matrix
        return_vector = df_25_ff_size_value_sorted_monthly[df_25_ff_size_value_sorted_monthly.date == date_to_consider].values[0, 1:].reshape(-1, 1)  # n*1  # there is not much difference between using demeaned returns or not_demeaned ones. I can replace df_25_ff_size_value_sorted_monthly with demeaned_return_df.
        

        # Compute realized returns
        return_of_simple_factor = (signal_vector @ return_vector)[0][0]
        realized_return_of_first_three_PP = (signal_vector @ first_n_PPs_position_matrix(U, VT, number_of_PPs_to_consider) @ return_vector)[0][0]
        expected_return_of_first_three_PP = first_n_PPs_expected_return(S, number_of_PPs_to_consider)
        realized_return_of_first_three_PEP = (signal_vector @ first_n_PEPs_position_matrix(eigenvectors,number_of_PEPs_to_consider) @ return_vector)[0][0]
        expected_return_of_first_three_PEP = first_n_PEPs_expected_return(eigenvalues, number_of_PEPs_to_consider)
        realized_return_of_first_three_PAP = (signal_vector @ first_n_PAPs_position_matrix(sorted_eigenvectors_ta_real_part,sorted_eigenvectors_ta_imaginary_part,number_of_PAPs_to_consider) @ return_vector)[0][0]
        expected_return_of_first_three_PAP = first_n_PAPs_expected_return(filtered_eigenvalues_ta, number_of_PAPs_to_consider)
        

        # Prepare a list for the current row values
        row_values = [
            date_index,
            return_of_simple_factor,  
            realized_return_of_first_three_PP, 
            expected_return_of_first_three_PP,
            realized_return_of_first_three_PEP,
            expected_return_of_first_three_PEP,
            realized_return_of_first_three_PAP,
            expected_return_of_first_three_PAP
        ]

        # Iterate over all Principal Portfolios (up to len(S)) and calculate realized/expected returns for each
        for i in range(len(S)):

            # for PP's
            realized_return_ith_PP = (signal_vector @ get_ith_position_matrix(U, VT, i) @ return_vector)[0][0]
            expected_return_ith_PP = get_ith_PPs_expected_return(S, i)
            # Add the values for realized and expected returns of the ith PP to the row
            row_values.append(realized_return_ith_PP)
            row_values.append(expected_return_ith_PP)

            # for PEP's
            realized_return_ith_PEP = (signal_vector @ get_ith_symmetric_position_matrix(eigenvectors, i) @ return_vector)[0][0]
            expected_return_ith_PEP = get_ith_PEPs_expected_return(eigenvalues, i)
            # Add the values for realized and expected returns of the ith PEP to the row
            row_values.append(realized_return_ith_PEP)
            row_values.append(expected_return_ith_PEP)


            # Dynamically add columns if they don't exist. for PP's.
            realized_col_name_pp = f"realized_return_of_{i+1}_PP"
            expected_col_name_pp = f"expected_return_of_{i+1}_PP"

            # Dynamically add columns if they don't exist. for PEP's.
            realized_col_name_pep = f"realized_return_of_{i+1}_PEP"
            expected_col_name_pep = f"expected_return_of_{i+1}_PEP"
            
            # for PP's
            if realized_col_name_pp not in realized_returns_df.columns:
                realized_returns_df[realized_col_name_pp] = None
            if expected_col_name_pp not in realized_returns_df.columns:
                realized_returns_df[expected_col_name_pp] = None

            #for PEP'S
            if realized_col_name_pep not in realized_returns_df.columns:
                realized_returns_df[realized_col_name_pep] = None
            if expected_col_name_pep not in realized_returns_df.columns:
                realized_returns_df[expected_col_name_pep] = None
                
        # The number of PAP's is different, so another loop is needed.
        for i in range(sorted_eigenvectors_ta_imaginary_part.shape[1]):
            # for PAP's
            realized_return_ith_PAP = (signal_vector @ get_ith_asymmetric_position_matrix(sorted_eigenvectors_ta_real_part,sorted_eigenvectors_ta_imaginary_part,i) @ return_vector)[0][0]
            expected_return_ith_PAP = get_ith_PAPs_expected_return(filtered_eigenvalues_ta,i)
            # Add the values for realized and expected returns of the ith PEP to the row
            row_values.append(realized_return_ith_PAP)
            row_values.append(expected_return_ith_PAP)

            # Dynamically add columns if they don't exist. for PEP's.
            realized_col_name_pap = f"realized_return_of_{i+1}_PAP"
            expected_col_name_pap = f"expected_return_of_{i+1}_PAP"

            #for PAP'S
            if realized_col_name_pap not in realized_returns_df.columns:
                realized_returns_df[realized_col_name_pap] = None
            if expected_col_name_pap not in realized_returns_df.columns:
                realized_returns_df[expected_col_name_pap] = None

        # Append the row to the dataframe
        realized_returns_df.loc[len(realized_returns_df)] = row_values

    realized_returns_df = realized_returns_df.set_index("date")

    pap_std = realized_returns_df['realized_return_of_first_three_PAP'].std()
    pep_std = realized_returns_df['realized_return_of_first_three_PEP'].std()
    realized_returns_df['adjusted_PAP'] = realized_returns_df['realized_return_of_first_three_PAP'] * (pep_std / pap_std)
    # Take the average of the adjusted "PAP" and "PEP"
    realized_returns_df['PEP and PAP 1-3'] = (realized_returns_df['adjusted_PAP'] + realized_returns_df['realized_return_of_first_three_PEP']) / 2
    # drop the adjusted column.
    realized_returns_df.drop(columns='adjusted_PAP', inplace=True)

    sharpe_df = realized_returns_df.drop(realized_returns_df.filter(like="expected").columns, axis=1).apply(lambda col: calculate_sharpe_ratio(col)) * math.sqrt(12)

    pp_columns = realized_returns_df.filter(like="PP")
    pp_realized_mean_df = pp_columns.filter(like="realized").mean(axis=0)
    pp_expected_mean_df = pp_columns.filter(like="expected").mean(axis=0)

    pep_columns = realized_returns_df.filter(like="PEP")
    pep_realized_mean_df = pep_columns.filter(like="realized").mean(axis=0)
    pep_expected_mean_df = pep_columns.filter(like="expected").mean(axis=0)

    pap_columns = realized_returns_df.filter(like="PAP")
    pap_realized_mean_df = pap_columns.filter(like="realized").mean(axis=0)
    pap_expected_mean_df = pap_columns.filter(like="expected").mean(axis=0)

    output_dict = {
    'realized_returns_df': realized_returns_df,
    'sharpe_df': sharpe_df,
    'pp_realized_mean_df': pp_realized_mean_df,
    'pp_expected_mean_df': pp_expected_mean_df,
    'pep_realized_mean_df': pep_realized_mean_df,
    'pep_expected_mean_df': pep_expected_mean_df,
    'pap_realized_mean_df': pap_realized_mean_df,
    'pap_expected_mean_df': pap_expected_mean_df
    }

    
    realized_returns_df, factor_data_monthly = filter_dataframes_by_common_dates(realized_returns_df,factor_data_monthly)
    X = factor_data_monthly[['Mkt-RF', 'SMB','HML','RMW','CMA']]
    Y = realized_returns_df['return_of_simple_factor']
    output_dict["regression_result_return_of_simple_factor"] = regression_results(X,Y)

    # Following Kelly's paper. For regressions that have PP's as Y, simple factor must be addes as X.
    X = pd.concat([X, realized_returns_df['return_of_simple_factor']], axis=1)
    # List of column names in realized_returns_df for which I want to perform the regression
    columns_to_iterate = ['realized_return_of_first_three_PP',
                      'realized_return_of_first_three_PEP',
                      'realized_return_of_first_three_PAP',
                       'PEP and PAP 1-3'] 

    # Iterate over each column
    for col in columns_to_iterate:
        # Set Y to the column from realized_returns_df
        Y = realized_returns_df[col]
        # Store the results with a variable name based on the column name
        result_name = f'regression_result_{col}'
        output_dict[result_name] = regression_results(X,Y)
    return output_dict  


In [2]:
df_25_ff_size_value_sorted_monthly = pd.read_csv(data_to_read_address)
df_25_ff_size_value_sorted_monthly['date'] = pd.to_datetime(df_25_ff_size_value_sorted_monthly['date'], format='%Y%m') + pd.offsets.MonthEnd(1)
df_25_ff_size_value_sorted_monthly.head(5)

,date,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
0,1926-07-31,5.8248,-1.7006,0.4875,-1.4580,2.0534,1.2077,2.4192,0.4926,-2.6049,...,1.5893,1.5278,1.2978,0.2727,2.4678,3.4539,6.0902,2.0266,3.1111,0.5623
1,1926-08-31,-2.0206,-8.0282,1.3796,1.4606,8.3968,2.3618,-1.1849,4.0084,0.5038,...,1.3336,3.8730,2.0021,2.1706,5.3422,1.0124,4.1903,2.0131,5.4849,7.7576
2,1926-09-30,-4.8291,-2.6154,-4.3417,-3.2729,0.8649,-2.6540,-1.2618,1.0829,-3.5480,...,1.0923,-0.5250,-1.7636,1.4646,0.8730,-1.2906,3.6538,0.0950,-0.7487,-2.4284
3,1926-10-31,-9.3729,-3.5519,-3.4948,3.4413,-2.5476,-2.8069,-3.2663,-5.0745,-8.0191,...,-3.3361,-2.6559,-2.1070,-3.1051,-5.3525,-2.7413,-3.0071,-2.2437,-4.6719,-5.8129
4,1926-11-30,5.5888,4.1877,2.4623,-4.4494,0.5362,3.1033,-2.3690,3.0078,5.1546,...,3.4448,2.3887,3.7335,4.9320,1.8213,4.2946,2.5326,1.5204,3.6619,2.5636


Note that I shift signals one period forward to make computations easier. 

In [3]:
signal_df = pd.DataFrame()
signal_df["date"] = df_25_ff_size_value_sorted_monthly["date"]
signal_df= signal_df.join(df_25_ff_size_value_sorted_monthly.iloc[:, 1:].shift(1))
signal_df.head()

,date,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
0,1926-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1926-08-31,5.8248,-1.7006,0.4875,-1.4580,2.0534,1.2077,2.4192,0.4926,-2.6049,...,1.5893,1.5278,1.2978,0.2727,2.4678,3.4539,6.0902,2.0266,3.1111,0.5623
2,1926-09-30,-2.0206,-8.0282,1.3796,1.4606,8.3968,2.3618,-1.1849,4.0084,0.5038,...,1.3336,3.8730,2.0021,2.1706,5.3422,1.0124,4.1903,2.0131,5.4849,7.7576
3,1926-10-31,-4.8291,-2.6154,-4.3417,-3.2729,0.8649,-2.6540,-1.2618,1.0829,-3.5480,...,1.0923,-0.5250,-1.7636,1.4646,0.8730,-1.2906,3.6538,0.0950,-0.7487,-2.4284
4,1926-11-30,-9.3729,-3.5519,-3.4948,3.4413,-2.5476,-2.8069,-3.2663,-5.0745,-8.0191,...,-3.3361,-2.6559,-2.1070,-3.1051,-5.3525,-2.7413,-3.0071,-2.2437,-4.6719,-5.8129


I can think of this matrix as $S_{t-1}$.

In [4]:
normalized_signal_df = rank_and_map(signal_df)
normalized_signal_df = normalized_signal_df[(normalized_signal_df['date'].dt.year > starting_year_to_filter) & (normalized_signal_df['date'].dt.year < end_year_to_filter)].reset_index(drop=True)
normalized_signal_df.head(5)

,date,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
0,1964-01-31,-0.500000,-0.375000,-0.333333,-0.458333,-0.416667,-0.291667,-0.250000,0.041667,0.000000,...,-0.208333,0.083333,0.208333,0.458333,0.500000,0.416667,0.333333,0.291667,0.375000,0.250000
1,1964-02-29,0.333333,-0.041667,0.416667,0.375000,0.500000,-0.500000,-0.458333,0.250000,-0.250000,...,-0.416667,-0.125000,-0.291667,0.208333,0.166667,0.000000,0.083333,-0.208333,-0.083333,-0.166667
2,1964-03-31,0.166667,0.083333,-0.083333,-0.458333,0.375000,-0.333333,-0.208333,0.041667,-0.125000,...,-0.041667,-0.291667,0.125000,0.500000,0.416667,-0.166667,-0.416667,-0.375000,0.291667,0.208333
3,1964-04-30,-0.416667,-0.250000,-0.166667,0.125000,-0.083333,-0.125000,0.041667,0.083333,0.375000,...,-0.208333,0.000000,0.291667,0.500000,-0.375000,-0.291667,-0.333333,0.208333,0.166667,-0.458333
4,1964-05-31,-0.250000,0.500000,0.125000,0.000000,-0.083333,-0.041667,-0.500000,0.291667,-0.291667,...,-0.458333,0.250000,0.416667,-0.416667,-0.375000,0.208333,0.375000,0.333333,-0.125000,0.458333


This matrix can be denoted as $R_{t-1}$

In [5]:
demeaned_return_df = cross_sectional_demean(df_25_ff_size_value_sorted_monthly)
demeaned_return_df = demeaned_return_df[(demeaned_return_df['date'].dt.year > starting_year_to_filter) & (demeaned_return_df['date'].dt.year < end_year_to_filter)].reset_index(drop=True)
demeaned_return_df.head()


,date,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
0,1964-01-31,1.613832,0.536932,1.849632,1.777732,2.481732,-4.392368,-2.864868,1.267032,-0.894568,...,-2.688568,-0.540368,-1.285868,1.231832,0.966432,0.730232,0.826532,-0.754868,-0.276068,-0.747368
1,1964-02-29,0.194676,0.090576,-0.496824,-2.474524,1.690076,-1.146724,-1.036524,-0.103724,-0.641224,...,-0.345224,-1.112324,0.098976,4.429176,1.845576,-0.934724,-2.243324,-1.563724,1.083376,0.986876
2,1964-03-31,-2.581656,-1.576556,-0.881556,0.463844,-0.262556,-0.741356,0.120944,0.271144,2.594944,...,-1.097756,-0.063256,2.110444,4.037044,-2.362856,-1.927556,-2.320556,0.729344,0.541144,-2.935256
3,1964-04-30,-1.015160,3.135640,0.094940,-0.361260,-0.374860,-0.361460,-3.334660,1.354440,-1.066060,...,-2.151560,0.884440,2.576040,-2.087360,-1.732360,0.781940,2.022140,1.358440,-0.412560,2.965240
4,1964-05-31,-2.800664,-2.863364,-1.009764,-1.201064,-0.119164,1.596436,-1.139464,1.466336,0.305336,...,-0.504664,0.772236,-0.103364,-0.204564,2.552136,0.523836,-1.212564,-0.175564,2.605636,1.837536


This gives: $R_{t}S'_{t}$

In [6]:
rs_matrix = compute_rs_product(demeaned_return_df, normalized_signal_df)

Prediction matrix for date T+1, used returns data up to month T and signals data up to month T-1. In the function get_prediction_matrix, I start the calculations from the previous month. Note that although the input date is the current data, but the in the function that month is excluded.

Note that in calculating realized returns, I am using the current month(the month of rearlized returns) as index. But remember that the matrix was $S_{t-1}$. So, the index actually retreives the value of the previous month. I formed the matrix this way in order to make the calculations easier.

In [7]:
# I leave out the first 120 observations to compute the prediction matrix.


realized_returns_df = pd.DataFrame(columns=[
    "date",
    "return_of_simple_factor", 
    "realized_return_of_first_three_PP", 
    "expected_return_of_first_three_PP",
    "realized_return_of_first_three_PEP",
    "expected_return_of_first_three_PEP",
    "realized_return_of_first_three_PAP",
    "expected_return_of_first_three_PAP"
])

for date_index in demeaned_return_df.iloc[number_of_lookback_periods:]['date']:
    date_to_consider = pd.Timestamp(date_index)
    
    #for PP's
    prediction_matrix = get_prediction_matrix(date_to_consider, rs_matrix, number_of_lookback_periods)
    U, S, VT = np.linalg.svd(prediction_matrix)

    #for PEP's
    Symmetric_prediction_matrix = (prediction_matrix + prediction_matrix.T)/2
    eigenvalues, eigenvectors = np.linalg.eig(Symmetric_prediction_matrix)
    idx = eigenvalues.argsort()[::-1]  # Sort in descending order
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # for PAP'S
    assymetric_prediction_matrix = 0.5 * (prediction_matrix - prediction_matrix.T)
    transposed_assymetric_prediction_matrix = assymetric_prediction_matrix.T
    eigenvalues_ta, eigenvectors_ta = np.linalg.eig(transposed_assymetric_prediction_matrix)
    sorted_indices_ta = np.argsort(-eigenvalues_ta.imag)
    sorted_eigenvalues_ta = eigenvalues_ta[sorted_indices_ta].imag
    sorted_eigenvectors_ta = eigenvectors_ta[:, sorted_indices_ta] * math.sqrt(2)  #sqrt(2) is to make the size of the vectors equal to 1.
    positive_indices = np.where(sorted_eigenvalues_ta > 0)
    filtered_eigenvalues_ta = sorted_eigenvalues_ta[positive_indices]
    filtered_eigenvectors_ta = sorted_eigenvectors_ta[:, positive_indices].squeeze()
    sorted_eigenvectors_ta_imaginary_part = filtered_eigenvectors_ta.imag
    sorted_eigenvectors_ta_real_part = filtered_eigenvectors_ta.real

    #to calculate realized returns
    signal_vector = normalized_signal_df[normalized_signal_df.date == date_to_consider].values[0, 1:].reshape(1, -1)  # 1*n matrix
    return_vector = df_25_ff_size_value_sorted_monthly[df_25_ff_size_value_sorted_monthly.date == date_to_consider].values[0, 1:].reshape(-1, 1)  # n*1  # there is not much difference between using demeaned returns or not_demeaned ones. I can replace df_25_ff_size_value_sorted_monthly with demeaned_return_df.
    

    # Compute realized returns
    return_of_simple_factor = (signal_vector @ return_vector)[0][0]
    realized_return_of_first_three_PP = (signal_vector @ first_n_PPs_position_matrix(U, VT, number_of_PPs_to_consider) @ return_vector)[0][0]
    expected_return_of_first_three_PP = first_n_PPs_expected_return(S, number_of_PPs_to_consider)
    realized_return_of_first_three_PEP = (signal_vector @ first_n_PEPs_position_matrix(eigenvectors,number_of_PEPs_to_consider) @ return_vector)[0][0]
    expected_return_of_first_three_PEP = first_n_PEPs_expected_return(eigenvalues, number_of_PEPs_to_consider)
    realized_return_of_first_three_PAP = (signal_vector @ first_n_PAPs_position_matrix(sorted_eigenvectors_ta_real_part,sorted_eigenvectors_ta_imaginary_part,number_of_PAPs_to_consider) @ return_vector)[0][0]
    expected_return_of_first_three_PAP = first_n_PAPs_expected_return(filtered_eigenvalues_ta, number_of_PAPs_to_consider)
    

    # Prepare a list for the current row values
    row_values = [
        date_index,
        return_of_simple_factor,  
        realized_return_of_first_three_PP, 
        expected_return_of_first_three_PP,
        realized_return_of_first_three_PEP,
        expected_return_of_first_three_PEP,
        realized_return_of_first_three_PAP,
        expected_return_of_first_three_PAP
    ]

    # Iterate over all Principal Portfolios (up to len(S)) and calculate realized/expected returns for each
    for i in range(len(S)):

        # for PP's
        realized_return_ith_PP = (signal_vector @ get_ith_position_matrix(U, VT, i) @ return_vector)[0][0]
        expected_return_ith_PP = get_ith_PPs_expected_return(S, i)
        # Add the values for realized and expected returns of the ith PP to the row
        row_values.append(realized_return_ith_PP)
        row_values.append(expected_return_ith_PP)

        # for PEP's
        realized_return_ith_PEP = (signal_vector @ get_ith_symmetric_position_matrix(eigenvectors, i) @ return_vector)[0][0]
        expected_return_ith_PEP = get_ith_PEPs_expected_return(eigenvalues, i)
        # Add the values for realized and expected returns of the ith PEP to the row
        row_values.append(realized_return_ith_PEP)
        row_values.append(expected_return_ith_PEP)


        # Dynamically add columns if they don't exist. for PP's.
        realized_col_name_pp = f"realized_return_of_{i+1}_PP"
        expected_col_name_pp = f"expected_return_of_{i+1}_PP"

        # Dynamically add columns if they don't exist. for PEP's.
        realized_col_name_pep = f"realized_return_of_{i+1}_PEP"
        expected_col_name_pep = f"expected_return_of_{i+1}_PEP"
        
        # for PP's
        if realized_col_name_pp not in realized_returns_df.columns:
            realized_returns_df[realized_col_name_pp] = None
        if expected_col_name_pp not in realized_returns_df.columns:
            realized_returns_df[expected_col_name_pp] = None

        #for PEP'S
        if realized_col_name_pep not in realized_returns_df.columns:
            realized_returns_df[realized_col_name_pep] = None
        if expected_col_name_pep not in realized_returns_df.columns:
            realized_returns_df[expected_col_name_pep] = None

    for i in range(sorted_eigenvectors_ta_imaginary_part.shape[1]):
        # for PAP's
        realized_return_ith_PAP = (signal_vector @ get_ith_asymmetric_position_matrix(sorted_eigenvectors_ta_real_part,sorted_eigenvectors_ta_imaginary_part,i) @ return_vector)[0][0]
        expected_return_ith_PAP = get_ith_PAPs_expected_return(filtered_eigenvalues_ta,i)
        # Add the values for realized and expected returns of the ith PEP to the row
        row_values.append(realized_return_ith_PAP)
        row_values.append(expected_return_ith_PAP)

        # Dynamically add columns if they don't exist. for PEP's.
        realized_col_name_pap = f"realized_return_of_{i+1}_PAP"
        expected_col_name_pap = f"expected_return_of_{i+1}_PAP"

        #for PAP'S
        if realized_col_name_pap not in realized_returns_df.columns:
            realized_returns_df[realized_col_name_pap] = None
        if expected_col_name_pap not in realized_returns_df.columns:
            realized_returns_df[expected_col_name_pap] = None

    # Append the row to the dataframe
    realized_returns_df.loc[len(realized_returns_df)] = row_values

realized_returns_df = realized_returns_df.set_index("date")

C:\Users\Amin\AppData\Local\Temp\ipykernel_4020\1428712266.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  realized_returns_df[expected_col_name_pep] = None
C:\Users\Amin\AppData\Local\Temp\ipykernel_4020\1428712266.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  realized_returns_df[realized_col_name_pap] = None
C:\Users\Amin\AppData\Local\Temp\ipykernel_4020\1428712266.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

In [8]:
pap_std = realized_returns_df['realized_return_of_first_three_PAP'].std()
pep_std = realized_returns_df['realized_return_of_first_three_PEP'].std()

realized_returns_df['adjusted_PAP'] = realized_returns_df['realized_return_of_first_three_PAP'] * (pep_std / pap_std)

# Step 2: Take the average of the adjusted "PAP" and "PEP"
realized_returns_df['PEP and PAP 1-3'] = (realized_returns_df['adjusted_PAP'] + realized_returns_df['realized_return_of_first_three_PEP']) / 2

# Optional: drop the adjusted column if you don't need it anymore
realized_returns_df.drop(columns='adjusted_PAP', inplace=True)


sharpe_df = realized_returns_df.drop(realized_returns_df.filter(like="expected").columns, axis=1).apply(lambda col: calculate_sharpe_ratio(col)) * math.sqrt(12)

pp_columns = realized_returns_df.filter(like="PP")
pp_realized_mean_df = pp_columns.filter(like="realized").mean(axis=0)
pp_expected_mean_df = pp_columns.filter(like="expected").mean(axis=0)

pep_columns = realized_returns_df.filter(like="PEP")
pep_realized_mean_df = pep_columns.filter(like="realized").mean(axis=0)
pep_expected_mean_df = pep_columns.filter(like="expected").mean(axis=0)

pap_columns = realized_returns_df.filter(like="PAP")
pap_realized_mean_df = pap_columns.filter(like="realized").mean(axis=0)
pap_expected_mean_df = pap_columns.filter(like="expected").mean(axis=0)

realized_returns_df.to_csv("temp/realized_returns.csv")
sharpe_df.to_csv("temp/sharpe.csv")

pp_columns.to_csv("temp/pp_columns.csv")
pp_realized_mean_df.to_csv("temp/pp_realized_mean_df.csv")
pp_expected_mean_df.to_csv("temp/pp_expected_mean_df.csv")

pep_columns.to_csv("temp/pep_columns.csv")
pep_realized_mean_df.to_csv("temp/pep_realized_mean_df.csv")
pep_expected_mean_df.to_csv("temp/pep_expected_mean_df.csv")

pap_columns.to_csv("temp/pap_columns.csv")
pap_realized_mean_df.to_csv("temp/pap_realized_mean_df.csv")
pap_expected_mean_df.to_csv("temp/pap_expected_mean_df.csv")

In [9]:
sharpe_df

return_of_simple_factor               0.415765
realized_return_of_first_three_PP     0.774135
realized_return_of_first_three_PEP    0.487988
realized_return_of_first_three_PAP    0.552814
realized_return_of_1_PP               0.481485
                                        ...   
realized_return_of_9_PAP             -0.056908
realized_return_of_10_PAP             0.015806
realized_return_of_11_PAP            -0.040131
realized_return_of_12_PAP             0.069891
PEP and PAP 1-3                       0.776921
Length: 67, dtype: float64

In [10]:
realized_returns_df, factor_data_monthly = filter_dataframes_by_common_dates(realized_returns_df,factor_data_monthly)

In [11]:
X = factor_data_monthly[['Mkt-RF', 'SMB','HML','RMW','CMA']]
X = pd.concat([X, realized_returns_df['return_of_simple_factor']], axis=1)
# Dependent variable (Y)
Y = realized_returns_df['realized_return_of_first_three_PP']

#Y = realized_returns_df['PEP and PAP 1-3']
#Y = realized_returns_df['return_of_simple_factor']
# Create and fit the model
X = sm.add_constant(X)

regression_results(X,Y)



[array([ 6.75901528,  0.02612368, -0.08107415,  0.19936143, -0.01035099,
        -0.10072436,  0.35732928]),
 array([ 4.34344091,  0.83669919, -1.78015137,  3.3844652 , -0.17170449,
        -1.07708357, 27.5893206 ]),
 0.6866793588839745,
 0.6008633838736177]

In [17]:
# List of column names in realized_returns_df for which I want to perform the regression
columns_to_iterate = ['realized_return_of_first_three_PP',
                      'realized_return_of_first_three_PEP',
                      'realized_return_of_first_three_PAP',
                       'PEP and PAP 1-3'] 

# Create a list to store the results
regression_results_list = []

# Iterate over each column
for col in columns_to_iterate:
    # Set Y to the column from realized_returns_df
    Y = realized_returns_df[col]
    
    # Store the results with a variable name based on the column name
    result_name = f'regression_result_{col}'
    regression_results_list.append((result_name, regression_results(X,Y)))



Results for regression_result_realized_return_of_first_three_PP:
0.6008633838736177
Results for regression_result_realized_return_of_first_three_PEP:
0.8921470173273893
Results for regression_result_realized_return_of_first_three_PAP:
0.050819266744861036
Results for regression_result_PEP and PAP 1-3:
0.4223392326177122
